# SHACL Simple Validation

Ocean Info Hub SHACL validation examples

It should be noted here that SHACL validation is not a service OIH offers.  Rather, 
the validation is a capacity that the OIH architectural approach facilities.  Further
this validation follows W3C recommendations as describted in 
[https://www.w3.org/TR/shacl/](https://www.w3.org/TR/shacl/).

* [SHACL Playground](https://shacl.org/playground/)
* [kglab SHACL validation with pySHACL](https://derwen.ai/docs/kgl/ex5_0/)


In [6]:
%%capture
!pip install pyshacl
!pip install 'PyLD>=2.0.3'
!pip install flatten_json
!pip install 'fsspec>=0.3.3'
!pip install s3fs
!pip install boto3
!pip install seaborn
!pip install dask
!pip install rdflib-jsonld
!pip install kglab
!pip install numba
!pip install pyld

In [3]:
import kglab as kg
from rdflib import Graph, plugin
from rdflib.serializer import Serializer

AttributeError: module 'pyarrow.lib' has no attribute '_CRecordBatchReader'

In [7]:
# set up the files we will use here
dg = './datagraphs/datagraph.json'
sg = './shapes/oih_learning.ttl'

In [8]:
import json
from pyld import jsonld
import os, sys

currentdir = os.path.dirname(os.path.abspath(''))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, currentdir)
from lib import jbutils

g = Graph().parse(sg, format='ttl')

context = {"schema": "https://schema.org/", 
           "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
           "shacl": "http://www.w3.org/ns/shacl#" ,
           "oihval": "https://oceans.collaborium.io/voc/validation/1.0.1/shacl#" 
           }

sgjld = g.serialize(format='json-ld', context=context, indent=4)

doc = json.loads(sgjld)

frame = {
  "@context": {"schema": "https://schema.org/", 
           "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
           "shacl": "http://www.w3.org/ns/shacl#" ,
           "oihval": "https://oceans.collaborium.io/voc/validation/1.0.1/shacl#" 
           },
  "@explicit": "false",
  "@requireAll": "true",
  "@type":     {},
  "shacl:message": "",
  "shacl:path": "",  
}

compacted = jsonld.compact(doc, context)

framed = jsonld.frame(compacted, frame)
jd = json.dumps(framed, indent=4)
# print(jd)

jbutils.show_graph(framed)

ModuleNotFoundError: No module named 'pyld'

In [210]:
# pyshack sends output to log along with the vars.  This suppresses that
import logging, sys
logging.disable(sys.maxsize)

In [213]:
import kglab

namespaces = {
    "schema":  "https://schema.org/",
    "shacl":   "http://www.w3.org/ns/shacl#" ,
    }

kg = kglab.KnowledgeGraph(
    name = "Schema.org based datagraph",
    base_uri = "https://example.org/id/",
    namespaces = namespaces,
    )

kg.load_jsonld(dg)

In [214]:
conforms, report_graph, report_text = kg.validate(
    shacl_graph=sg,
    shacl_graph_format="ttl"
)

In [215]:
import pandas as pd
pd.set_option("max_rows", None)
        
sparql = """
SELECT ?path ?value ?constraint ?message ?id ?focus 
  WHERE {
    ?id rdf:type shacl:ValidationResult .
    ?id shacl:focusNode ?focus .
    ?id shacl:resultMessage ?message .
    ?id shacl:sourceConstraintComponent ?constraint .
    OPTIONAL {
        ?id shacl:resultPath ?path .
    }
    OPTIONAL {
        ?id shacl:value ?value .
    }
  }
"""

df = report_graph.query_as_df(sparql)
df.head(10)

,path,constraint,message,id,focus,value
0,schema:provider,shacl:MinCountConstraintComponent,A provider must be noted,_:nc0c23de19cda41839d796f82d56c39b4b2,_:nc0c23de19cda41839d796f82d56c39b4b3,NaN
1,NaN,shacl:NodeKindConstraintComponent,Graph must have an ID,_:nc0c23de19cda41839d796f82d56c39b4b4,_:nc0c23de19cda41839d796f82d56c39b4b3,_:nc0c23de19cda41839d796f82d56c39b4b3
2,schema:name,shacl:MinCountConstraintComponent,Name is required,_:nc0c23de19cda41839d796f82d56c39b4b5,_:nc0c23de19cda41839d796f82d56c39b4b3,NaN
3,schema:url,shacl:MinCountConstraintComponent,URL required for the location of the resource ...,_:nc0c23de19cda41839d796f82d56c39b4b6,_:nc0c23de19cda41839d796f82d56c39b4b3,NaN


In [205]:

VIS_STYLE = {
    "schema": {
        "color": "green",
        "size": 20,
    },
    "shacl":{
        "color": "red",
        "size": 20,
    },
    "_":{
        "color": "orange",
        "size": 20,
    },
}

subgraph = kglab.SubgraphTensor(report_graph)
pyvis_graph = subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)
pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig05.html")